# Exercice 1

Build up your own python program to solve a beam with an arbritary number of elements and arbritary loading. Do it either in a text file (`.py`) or in a new jupyther notebook. You can use object-oriented programming, as suggested below.

You find below an example to start a `Frame` class. The example include geometry and visualization features. You should add the *methods* (*i.e.* the functions of the class) to assemble `K`, `F`, impose bcs and solve the system.

Once done, test the solution for a cantilever beam under its own weigth. Compare againt the analytical solution

In [14]:
import matplotlib.pyplot as plt 
import numpy as np

class Frame():
    
    def __init__(self, nodes, elements):
        self.nodes = nodes
        self.elements = elements
        self.ndof = 3*self.nodes.size
        self.U = np.zeros(self.ndof)
        print("%i ndof"%self.ndof)
    
    def L(self,e):
        dx = self.nodes[self.elements[3]][1]-self.nodes[self.elements[3]][0]
        return np.sqrt(dx[0]**2+dx[1]**2)
        
    def dof_map_e(self, e):
        return np.concatenate((frame.elements[e,0]*3+np.arange(3),frame.elements[e,1]*3+np.arange(3)))
    
    def dof_map(self, e, i_local):
        return self.dof_map_e(e)[i_local]
    
    def U_e(self,e):
        return [self.U[self.dof_map(e,i)] for i in range(6)]

    def S(self, s, e):
        L = self.L(e)
        S = np.array([[1 - s/L, 0, 0, s/L, 0, 0], 
                [0, 1 - 3*s**2/L**2 + 2*s**3/L**3, s - 2*s**2/L + s**3/L**2, 0, 3*s**2/L**2 - 2*s**3/L**3, -s**2/L + s**3/L**2]])
        return S
        
    def uv(self,e,s): 
        return np.dot(frame.S(s,e),frame.U_e(e))
    
    def xy(self,e,s):
        element = self.elements[e]
        x0coords = self.nodes[element][:,0]
        y0coords = self.nodes[element][:,1]
        x0 = x0coords[0]*(1-s/self.L(e))+x0coords[1]*s/self.L(e)
        y0 = y0coords[0]*(1-s/self.L(e))+y0coords[1]*s/self.L(e)
        [u,v] = self.uv(e,s)
        return [x0+u, y0+v]
    
    def plot(self):
        #import matplotlib.plt as plt 
        fig, ax = plt.subplots()
        shift = self.nodes.max()*.005
        for (e_num,e) in enumerate(self.elements):
            xcoords = self.nodes[e][:,0]
            ycoords = self.nodes[e][:,1]
            ax.plot(xcoords, ycoords,'o-',lw=2, color='black', ms=10)
        ax.set_xlim(self.nodes[:,0].min()-10*shift, self.nodes[:,0].max()+10*shift)
        ax.set_ylim(self.nodes[:,1].min()-10*shift, self.nodes[:,1].max()+10*shift)
        return ax
    
    def plot_with_label(self):
        fig, ax = plt.subplots()
        shift = self.nodes.max()*.005
        for (e_num,e) in enumerate(self.elements):
            xcoords = self.nodes[e][:,0]
            ycoords = self.nodes[e][:,1]
            ax.plot(xcoords, ycoords,'o-',lw=2, color='black', ms=10)
            ax.text((xcoords[0]+xcoords[1])/2.+shift, (ycoords[0]+ycoords[1])/2.+shift, str(e_num), bbox=dict(facecolor='yellow'))
            for i in range(e.size):
                ax.text(xcoords[i]+shift, ycoords[i]+shift, e[i], verticalalignment = 'bottom', horizontalalignment = 'left')
        ax.set_xlim(self.nodes[:,0].min()-10*shift, self.nodes[:,0].max()+10*shift)
        ax.set_ylim(self.nodes[:,1].min()-10*shift, self.nodes[:,1].max()+10*shift)
        return ax

    def plot_displaced(self, U): 
        for e in range(4):
            sv = np.linspace(0,frame.L(e),10)
            plt.plot([frame.xy(e,s)[0] for s in sv],[frame.xy(e,s)[1] for s in sv],'b.-')
            plt.plot(self.nodes.transpose(),'o')
            

An example

In [15]:
nodes = np.array([[ 0.  ,  0.  ],
       [ 0.25,  0.  ],
       [ 0.5 ,  0.  ],
       [ 0.75,  0.  ],
       [ 1.  ,  0.  ]])
elements = np.array([[0, 1],
       [1, 2],
       [2, 3],
       [3, 4]])
Usol = np.array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -3.75000000e-01,  -5.00000000e-01,
         0.00000000e+00,  -6.66666667e-01,   4.93432455e-17,
         0.00000000e+00,  -3.75000000e-01,   5.00000000e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])

frame = Frame(nodes,elements)   
frame.plot_with_label()
frame.U = .05*Usol
frame.plot_displaced(Usol)

30 ndof


# Exercice 2

Extend the `Frame` class by including the case of beam segments with several orientations. Look also at the reference in the Syllabus. Test on examples